In [10]:
import pandas as pd
import pickle as pkl
import numpy as np

import sys
sys.path.append('../')
from utils import *

In [ ]:
embeddings = pd.read_pickle('../../data/embeddings/title-autoencoded')
meta_data = pd.read_pickle('../../data/videos_raw_metadata')

In [41]:
df = pd.read_parquet('../../logs.parquet')
# df = df[df['is_video']]
users = list(set(df['userId'].tolist()))

In [42]:
videos = clean(meta_data, titles=embeddings)

In [59]:
data = dict()

for user in tqdm(users):
    data[user] = df[df['userId'] == user].sort_values('date')

sessions = dict()
for user in tqdm(users):
    sessions[user] = []
    sess = data[user][['date', 'url', 'is_video', 'upnext_content']].to_dict('records')
    session = []
    session_active = False
    for watch in sess:
        # when waiting for first video in a session
        if not session_active and watch['is_video']:
            session = []
            session.append(watch)
            session_active = True

        # back to back videos
        elif session_active and watch['is_video']:
            upnexts = decode_upnext(session[-1]['upnext_content'])
            this_url = get_id(watch['url'])
            if this_url in upnexts:
                session.append(watch)


        # if next watch is not video when session active
        elif session_active and not watch['is_video']:
            session_active = False
            if len(session) > 1:
                sessions[user].append(session)
            session = []

  0%|          | 0/2171 [00:00<?, ?it/s]

In [65]:
session_users = []
for user in sessions:
    if len(sessions[user]) != 0:
        session_users.append(user)

In [68]:
watches = []
for user in tqdm(session_users):
    for session in sessions[user]:
        for watching, next_watch in zip(session[:-1], session[1:]):
            watch = dict()
            watch['playing'] = get_id(watching['url']) 
            watch['upnext'] = decode_upnext(watching['upnext_content']) 
            watch['selected'] = get_id(next_watch['url'])
            watches.append(watch)

  0%|          | 0/1289 [00:00<?, ?it/s]

In [72]:
pkl.dump(watches, open('../../data/datasets/raw-video-level-watches', 'wb'))